In [ ]:
!pip install physbo

In [ ]:
import numpy as np

import physbo
import matplotlib.pyplot as plt
import random


# Onemax prolém

In [ ]:
N = 10
X = []

for i in range(2**N):
  g = []
  for j in range(N):
    g.append(i >> j & 1)
  X.append(g)

# task  - uložte do X všechny možné pole binárních hodnot délky N


In [ ]:
plt.plot([sum(s) for s in X])

In [ ]:
test_X = np.array(random.sample(X,len(X)))

In [ ]:
plt.plot([sum(s) for s in test_X])

In [ ]:
policy = physbo.search.discrete.policy(test_X=test_X)

In [ ]:
# ÚKOL - prozkoumejte proměnou test_X

In [ ]:
# ÚKOL
# napište simulátor, který vrací sum všech čísel, možno i pro více samplů naráz

def simulator(actions:int):
    fx = 0 #TODO
    return fx

In [ ]:
#policy.set_seed(3)

policy.random_search(max_num_probes=3, simulator=simulator)

In [ ]:
# Jak to vypadá?
best_fx, best_actions = policy.history.export_sequence_best_fx()
print(f"best_fx: {best_fx[-1]} at {test_X[best_actions[-1]]}")

In [ ]:
res = policy.bayes_search(max_num_probes=4, simulator=simulator, score="EI",
    interval=1, num_rand_basis=500)
best_fx, best_actions = policy.history.export_sequence_best_fx()
print(f"best_fx: {best_fx[-1]} at {test_X[best_actions[-1]]}")

In [ ]:
# jak to hledalo?

fig = plt.figure()
plt.plot(res.fx[0:res.total_num_search])
best_fx, best_action = res.export_all_sequence_best_fx()
plt.plot(best_fx)

In [ ]:
# kde hledat?
# Úkol - zkuste jiné acquisition funkce
scores = policy.get_score(mode="EI", xs=test_X)
fig = plt.figure()
plt.plot(scores)

In [ ]:
fig = plt.figure()
plt.plot([np.sum(x) for x in test_X])


# A teď jiná funkce

In [ ]:
def simulator(actions:int) -> float:
    x = alpha_val[actions][0]
    fx = 0.1 * ( 2 * x - 1 ) * ( x - 3 ) * ( x - 5 )
    alpha_action.append(x)
    fx_action.append(fx)
    return fx





In [ ]:
# Úkol: zobrazte graf této funkce

In [ ]:

window_num=10001
alpha_max = 5.0
alpha_min = 0.0
alpha_action = []
fx_action = []
alpha_val = np.linspace(alpha_min,alpha_max,window_num).reshape(window_num, 1)



policy = physbo.search.discrete.policy(test_X=alpha_val)
policy.set_seed(10)
policy.random_search(max_num_probes=1, simulator=simulator)
policy.bayes_search(max_num_probes=1, simulator=simulator, score="EI", interval=1, num_rand_basis=500)

In [ ]:
mean = policy.get_post_fmean(alpha_val)
var  = policy.get_post_fcov(alpha_val)

In [ ]:

# Zobrazení funkce

def plotfig(policy, X):
    mean = policy.get_post_fmean(X)
    var = policy.get_post_fcov(X)
    std = np.sqrt(var)

    x = X[:,0]
    fig, ax = plt.subplots()
    ax.plot(x, mean)
    ax.fill_between(x, (mean-std), (mean+std), color='b', alpha=.1)
    ax.scatter(alpha_action, fx_action)
    x1 = np.arange(0, 5, 0.01)
    y1 = 0.1 * ( 2 * x1 - 1 ) * ( x1 - 3 ) * ( x1 - 5 )
    plt.plot(x1, y1, color='#ff4500')
    plt.show()


In [ ]:
policy.bayes_search(max_num_probes=10, simulator=simulator, score="TS", interval=1, num_rand_basis=500)
plotfig(policy, alpha_val)

In [ ]:
scores = policy.get_score(mode="TS", xs=alpha_val)
fig = plt.figure()
plt.plot(scores)

# Více parametrů pro optimalizaci

**Pareto optimalita**: Řešení je Pareto optimální, pokud není možné zlepšit jeden cíl bez zhoršení alespoň jednoho jiného cíle. (Vilfredo Pareto - ekonom)

Existuje více Pareto optimálních řešení. Tyto body představují kompromisy mezi různými cíli.

In [ ]:
import itertools
a = np.linspace(-2,2,101)
test_X = np.array(list(itertools.product(a, a)))

In [ ]:
# Úkol Prozkoumejte text_X


In [ ]:
# Úkol - jak funguje tato funkce?

def vlmop2_minus(x):
    n = x.shape[1]
    y1 = 1 - np.exp(-1 * np.sum((x - 1/np.sqrt(n)) ** 2, axis = 1))
    y2 = 1 - np.exp(-1 * np.sum((x + 1/np.sqrt(n)) ** 2, axis = 1))

    return np.c_[-y1, -y2]

In [ ]:
class simulator(object):
    def __init__(self, X):
        self.t = vlmop2_minus(X)

    def __call__( self, action):
        return self.t[action]

In [ ]:
simu = simulator(test_X)

In [ ]:
plt.figure()
plt.imshow(simu.t[:,0].reshape((101,101)), vmin=-1.0, vmax=0.0, origin="lower", extent=[-2.0, 2.0, -2.0, 2.0])
plt.title("objective 1")
plt.colorbar()
plt.plot([1.0/np.sqrt(2.0)], [1.0/np.sqrt(2.0)], '*')
plt.show()

In [ ]:
# ÚKOL: zobrazte objective 2

In [ ]:
policy = physbo.search.discrete_multi.policy(test_X=test_X, num_objectives=2)
policy.set_seed(0)

In [ ]:
policy = physbo.search.discrete_multi.policy(test_X=test_X, num_objectives=2)
policy.set_seed(0)

res_random = policy.random_search(max_num_probes=50, simulator=simu)

In [ ]:
res_random = policy.random_search(max_num_probes=50, simulator=simu, disp_pareto_set=True)

In [ ]:
front, front_num = res_random.export_pareto_front()
front, front_num

In [ ]:
def plot_pareto_front(res):
    front, front_num = res.export_pareto_front()
    dominated = [i for i in range(res.num_runs) if i not in front_num]
    points = res.fx[dominated, :]

    plt.figure(figsize=(7, 7))
    plt.scatter(res.fx[dominated,0], res.fx[dominated,1], c = "blue")
    plt.scatter(front[:, 0], front[:, 1], c = "red")
    plt.title('Pareto front')
    plt.xlabel('Objective 1')
    plt.ylabel('Objective 2')
    plt.xlim([-1.0,0.0])
    plt.ylim([-1.0,0.0])

plot_pareto_front(res_random)